In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
import molpack as mpk

In [2]:
H2O = mp.Struct(
    "h2o",
    atoms=[
        mp.Atom(name="O", x=0.00000, y=-0.06556, z=0.00000),
        mp.Atom(name="H1", x=0.75695, y=0.52032, z=0.00000),
        mp.Atom(name="H2", x=-0.75695, y=0.52032, z=0.00000),
    ],
    bonds=[],
)
H2O.add_bond_("O", "H1")
H2O.add_bond_("O", "H2")

<Struct 3 atoms>

In [3]:
targets = [mpk.Target(H2O.to_frame(), 10, mp.region.Cube(np.array([0, 0, 0]), 10))]

In [4]:
import nlopt

In [ ]:
n = 5
molecule_coords = [
    H2O.to_frame()["atoms"][["x", "y", "z"]].to_numpy() for _ in range(n)
]

d_min = 1.0


def f(x, grad):
    penalty = 0.0
    translations = x[: 3 * n].reshape(n, 3)
    rotations = x[3 * n :].reshape(n, 4)

    for i in range(n):
        for j in range(i + 1, n):
            min_dist = np.inf
            for k1 in range(len(molecule_coords[i])):
                for k2 in range(len(molecule_coords[j])):
                    coord_i = (
                        mp.op.rotate_q(molecule_coords[i][k1], rotations[i])
                        + translations[i]
                    )
                    coord_j = (
                        mp.op.rotate_q(molecule_coords[j][k2], rotations[j])
                        + translations[i]
                    )
                    dist = np.linalg.norm(coord_i - coord_j)
                    min_dist = min(min_dist, dist)
            if min_dist < d_min:
                penalty += (d_min - min_dist) ** 2 # + geometry penalty
    print(penalty)
    return penalty


opt = nlopt.opt(nlopt.LD_SLSQP, 7 * 5)

opt.set_min_objective(f)
lower_bounds = np.hstack(
    [-10 * np.ones(3 * n), -1 * np.ones(4 * n)]
)  # 平移下界 & 四元数下界
upper_bounds = np.hstack(
    [10 * np.ones(3 * n), 1 * np.ones(4 * n)]
)  # 平移上界 & 四元数上界
opt.set_lower_bounds(lower_bounds)
opt.set_upper_bounds(upper_bounds)

opt.set_xtol_rel(1e-6)

# 开始优化
initial_translations = np.zeros((n, 3))  # 初始平移
initial_rotations = np.array([[1, 0, 0, 0] for _ in range(n)])  # 初始单位四元数
initial_guess = np.hstack([initial_translations.flatten(), initial_rotations.flatten()])
optimized_result = opt.optimize(initial_guess)

# 提取结果
optimized_translations = optimized_result[: 3 * n].reshape(n, 3)
optimized_rotations = optimized_result[3 * n :].reshape(n, 4)

10.0


In [23]:
upper_bounds

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
       10., 10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [10]:
opt_val = opt.last_optimum_value()
result = opt.last_optimize_result()

In [11]:
opt_val

10.0

In [12]:
result

4